In [ ]:
import numpy as np
import heapq
import math
import random

In [ ]:
# Clase Nodo para representar cada símbolo en el árbol de Huffman
class Nodo:
    def __init__(self, simbolo=None, frecuencia=None):
        self.simbolo = simbolo
        self.frecuencia = frecuencia
        self.izquierda = None
        self.derecha = None

    # Método para comparar nodos según su frecuencia (necesario para la cola de prioridad)
    def __lt__(self, otro):
        return self.frecuencia < otro.frecuencia

In [ ]:
# Función para construir el árbol de Huffman
def construir_arbol_huffman(simbolos, frecuencias):
    # Crear una cola de prioridad inicial con nodos para cada símbolo
    cola_prioridad = [Nodo(simbolo, frecuencia) for simbolo, frecuencia in zip(simbolos, frecuencias)]
    heapq.heapify(cola_prioridad)  # Convertir la lista en una cola de prioridad

    # Combinar nodos hasta que quede un único árbol
    while len(cola_prioridad) > 1:
        # Extraer los dos nodos con menor frecuencia
        hijo_izquierdo = heapq.heappop(cola_prioridad)
        hijo_derecho = heapq.heappop(cola_prioridad)
        # Crear un nuevo nodo combinando las frecuencias
        nodo_combinado = Nodo(frecuencia=hijo_izquierdo.frecuencia + hijo_derecho.frecuencia)
        nodo_combinado.izquierda = hijo_izquierdo
        nodo_combinado.derecha = hijo_derecho
        heapq.heappush(cola_prioridad, nodo_combinado)  # Añadir el nuevo nodo a la cola

    # Retornar la raíz del árbol
    return cola_prioridad[0]

In [ ]:
# Función para generar los códigos Huffman
def generar_codigos_huffman(nodo, codigo="", codigos_huffman={}):
    if nodo is not None:
        # Si el nodo es una hoja, asignar el código al símbolo
        if nodo.simbolo is not None:
            codigos_huffman[nodo.simbolo] = codigo
        # Recursivamente generar códigos para los hijos izquierdo y derecho
        generar_codigos_huffman(nodo.izquierda, codigo + "0", codigos_huffman)
        generar_codigos_huffman(nodo.derecha, codigo + "1", codigos_huffman)

    return codigos_huffman

In [ ]:
# Función para decodificar un mensaje codificado con Huffman usando el árbol de Huffman
def decodificar_huffman(mensaje_codificado, raiz):
    mensaje_decodificado = ""
    nodo_actual = raiz

    for bit in mensaje_codificado:
        # Recorrer el árbol según el bit actual
        if bit == '0':
            nodo_actual = nodo_actual.izquierda
        else:
            nodo_actual = nodo_actual.derecha

        # Si llegamos a un nodo hoja, agregar el símbolo al mensaje decodificado
        if nodo_actual.izquierda is None and nodo_actual.derecha is None:
            mensaje_decodificado += nodo_actual.simbolo
            nodo_actual = raiz  # Reiniciar para el siguiente símbolo

    return mensaje_decodificado

In [ ]:
# Función para agregar errores aleatorios al mensaje codificado
def agregar_errores_aleatorios(mensaje_codificado, tasa_error=0.1):
    vector_error = ''.join('1' if random.random() < tasa_error else '0' for _ in mensaje_codificado)
    # Realizar XOR entre el mensaje codificado y el vector de error para alterar bits
    mensaje_alterado = ''.join('1' if mensaje_codificado[i] != vector_error[i] else '0' for i in range(len(mensaje_codificado)))
    return mensaje_alterado, vector_error

# Códigos Huffman

In [ ]:
import numpy as np
import heapq
import random
from collections import Counter

# ===== Ejemplo dado =====
#Cambiar la frase que me den
frase = "abcdfeacbdefabcdefadcbefcbbdaef"

# ===== 2.a) Símbolos y frecuencias AUTOMÁTICO =====
def simbolos_y_frecuencias_desde_frase(frase):
    conteo = Counter(frase)  # {simbolo: veces}
    # Ordenar por símbolo para que sea reproducible (opcional)
    simbolos = sorted(conteo.keys())
    frecuencias = [conteo[s] for s in simbolos]
    return simbolos, frecuencias

simbolos, frecuencias = simbolos_y_frecuencias_desde_frase(frase)

print("Símbolos:", simbolos)
print("Frecuencias:", frecuencias)
print("Total símbolos:", sum(frecuencias))

# ===== Construir el árbol de Huffman =====
raiz = construir_arbol_huffman(simbolos, frecuencias)

# ===== Generar los códigos Huffman =====
codigos_huffman = generar_codigos_huffman(raiz, codigo="", codigos_huffman={})

# ===== Imprimir los códigos Huffman =====
print("\nCódigos Huffman:")
for s in sorted(codigos_huffman.keys()):
    print(f"Símbolo: {s}, Código: {codigos_huffman[s]}")

# ===== 2.c) Longitud promedio Huffman =====
N = len(frase)
probabilidades = {s: frecuencias[i] / N for i, s in enumerate(simbolos)}
longitudes = {s: len(codigos_huffman[s]) for s in simbolos}

longitud_huffman = sum(probabilidades[s] * longitudes[s] for s in simbolos)
print(f"\nLongitud promedio Huffman: {longitud_huffman:.4f} bits/símbolo")

# ===== 2.d) Codificación de longitud fija =====
# bits por símbolo = ceil(log2(|alfabeto|))
L_fija = int(np.ceil(np.log2(len(simbolos))))
print(f"Longitud fija mínima: {L_fija} bits/símbolo")

# ===== 2.c) Entropía de la fuente =====
entropia = -sum(probabilidades[s] * np.log2(probabilidades[s]) for s in simbolos)
print(f"Entropía de la fuente: {entropia:.4f} bits/símbolo")

# Comparación y conclusión
print("\nComparación:")
print(f"Entropía H = {entropia:.4f}")
print(f"Longitud media Huffman L̄_H = {longitud_huffman:.4f}")
print(f"Longitud fija L_fija = {L_fija:.4f}")
print(f"Ineficiencia Huffman (L̄_H - H) = {longitud_huffman - entropia:.4f} bits/símbolo")
print("Huffman suele cumplir: H <= L̄_H < H + 1")

# ===== 2.e) Codificar y decodificar SIN errores =====
mensaje_codificado = ''.join(codigos_huffman[ch] for ch in frase)
print("\nMensaje codificado (Huffman):", mensaje_codificado)

mensaje_decodificado = decodificar_huffman(mensaje_codificado, raiz)
print("Mensaje decodificado (sin errores):", mensaje_decodificado)
print("¿Coincide con la frase original?", mensaje_decodificado == frase)

# ===== 2.e) Agregar errores aleatorios y decodificar =====
tasa_error = 0.1  # 10% de errores
mensaje_alterado, vector_error = agregar_errores_aleatorios(mensaje_codificado, tasa_error)

print("\nVector de error:", vector_error)
print("Mensaje alterado (Huffman):", mensaje_alterado)

mensaje_decodificado_err = decodificar_huffman(mensaje_alterado, raiz)
print("Mensaje decodificado (con errores):", mensaje_decodificado_err)

# Métrica sencilla: cuántos símbolos coinciden (hasta la longitud mínima)
m = min(len(frase), len(mensaje_decodificado_err))
coinciden = sum(1 for i in range(m) if frase[i] == mensaje_decodificado_err[i])
print(f"Coincidencias (posición a posición, hasta min long): {coinciden}/{m}")




Símbolos: ['a', 'b', 'c', 'd', 'e', 'f']
Frecuencias: [5, 6, 5, 5, 5, 5]
Total símbolos: 31

Códigos Huffman:
Símbolo: a, Código: 110
Símbolo: b, Código: 01
Símbolo: c, Código: 101
Símbolo: d, Código: 00
Símbolo: e, Código: 111
Símbolo: f, Código: 100

Longitud promedio Huffman: 2.6452 bits/símbolo
Longitud fija mínima: 3 bits/símbolo
Entropía de la fuente: 2.5814 bits/símbolo

Comparación:
Entropía H = 2.5814
Longitud media Huffman L̄_H = 2.6452
Longitud fija L_fija = 3.0000
Ineficiencia Huffman (L̄_H - H) = 0.0638 bits/símbolo
Huffman suele cumplir: H <= L̄_H < H + 1

Mensaje codificado (Huffman): 1100110100100111110101010011110011001101001111001100010101111100101010100110111100
Mensaje decodificado (sin errores): abcdfeacbdefabcdefadcbefcbbdaef
¿Coincide con la frase original? True

Vector de error: 1000000000000000100010000000000101000000010000000001000000000000000001000000010010
Mensaje alterado (Huffman): 010011010010011101011101001111011000110101111100110101010111110010101110011

In [ ]:
#Repetir el apartado anterior considerando el siguiente mensaje "Que es eso eso es queso"